# Project Details

**Batch**                -  May 2019 Batch (Group 10 A)  
**Project Type**         -  Capstone project  
**Project Domain**       -  NLP  
**Project Name**         -  Automatic Ticket Assignment  
**Submission Date**      -  17-May-2020      
**Submitted By**         -  Group10A  
**Delivery Type**        -  Milestone 2,3

## The Real Problem
One of the key activities of any IT function is to “Keep the lights on” to ensure there is noimpact to the Business operations. IT leverages Incident Management process to achieve theabove Objective. An incident is something that is unplanned interruption  to  an  IT  service  orreduction  in  the quality of an IT service that affects the Users and theBusiness. The main goal of Incident Management process is to provide a quick fix / workarounds or solutions thatresolves the interruption and restores the service to its full capacity to ensure no businessimpact.In most of the organizations, incidents are created by various Business and IT Users, End Users/ Vendors if they have access to ticketing systems, and from the integrated monitoringsystems and tools. Assigning the incidents to the appropriate person or unit in the support team has critical importance to provide improved user satisfaction while ensuring better allocation of support resources. The assignment of incidents to appropriate IT 
groups is still a manual process in many of the IT organizations.Manual assignment of incidents is time consuming and requires human efforts. There may bemistakes due to human errors and resource consumption is carried out ineffectively because of the misaddressing. On the other hand, manual assignment increases the response and resolution times which result in user satisfaction deterioration / poor customer service.

# Business Domain Value
In the support process,incoming incidents are analyzed and assessed by organization’s support teams to fulfill the request. 
In many organizations, better allocation and effective usage of the valuable support resources will directly result 
in substantial cost savings.Currently the incidents are created by various stakeholders (Business Users, IT Users 
and Monitoring Tools) within IT Service Management Tool and are assigned to Service Desk teams (L1 / L2 teams). This team will review the incidents for right ticket categorization, priorities and then carry out initial diagnosis to see if they can resolve. Around ~54% of the incidents are resolved by L1 / L2 teams. Incase L1 / L2 is unable to resolve, they will then escalate / assign the tickets to Functional teams from Applications and Infrastructure (L3 teams). Some portions of incidents are directly assigned to L3 teams by either Monitoring tools or Callers / Requestors. L3 teams will carry out detailed diagnosis and resolve the incidents. Around ~56% 
of incidents are resolved by Functional / L3 teams. Incase if vendor support is needed, they will reach out for their support towards incident closure.L1 / L2 needs to spend time reviewing Standard Operating Procedures (SOPs) before assigning to Functional teams (Minimum ~25-30% of incidents needs to be reviewed for SOPs before ticket assignment). 15 min is being spent for SOP review for each incident. Minimum of ~1 FTE effort needed only for incident assignment to L3 teams.During the process of incident assignments by L1 / L2 teams to functional groups, there were multiple instances of incidents getting assigned to wrong functional groups. Around ~25% of Incidents are wrongly assigned to functional teams. Additional effort needed for Functional teams to re-assign to right functional groups. During this process, some of the incidents are in queue and not addressed timely resulting in poor customer service.Guided by powerful AI techniques that can classify incidents to right functional groups can help organizations to reduce the resolving time of the issue and can focus on more productive tasks.

# Project Description

In this capstone project, the goal is to build a classifier that can classify the tickets by analysing text.Details about the data and dataset files are given in below link,https://drive.google.com/file/d/1OZNJm81JXucV3HmZroMq6qCT2m7ez7IJ 

## Milestone 1: 
● Pre-Processing, Data Visualisation and EDA Overview  
● Exploring the given Data files  
● Understanding the structure of data  
● Missing points in data  
● Finding inconsistencies in the data  
● Visualizing different patterns   
● Visualizing different text features  
● Dealing with missing values  
● Text preprocessing   
● Creating word vocabulary from the corpus of report text data  
● Creating tokens as required   

## Milestone 2
● Model BuildingOverview  
● Building a model architecture which can classify.  
● Trying different model architectures by researching state of the art for similar tasks.  
● Train the model  
● To deal with large training time, save the weights so that you can use them when training the model for the second time without starting from scratch.  

## Milestone 3:  
● Test the Model, Fine-tuning and RepeatOverview.  
● Test the model and report as per evaluation metrics.  
● Try different models.  
● Try different evaluation metrics.   
● Set different hyper parameters, by trying different optimizers, loss functions, epochs, learning rate, batch size, checkpointing, early stopping etc..for these models to fine-tune them.  
● Report evaluation metrics for these models along with your observation on how changing different hyper parameters leads to change in the final evaluation metric.  

In [11]:
# # Let us first start with the date to note down timing
# import warnings
# warnings.filterwarnings('always')

from datetime import datetime
def time_now():
    return datetime.now().strftime("%H:%M:%S")

def print_msg(*msg):
    print(time_now(),":",*msg)

## Import all the required modules
print_msg("importing modules")

## most used packages
import numpy as np
import pandas as pd
import re,os,io,json

### NLP packages
import nltk
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer

## Keras packages
from tensorflow.keras.preprocessing.text import Tokenizer,tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input ,Flatten,BatchNormalization
from tensorflow.keras.layers import Embedding,GlobalMaxPool1D,Bidirectional,SpatialDropout1D
from tensorflow.keras.models import Sequential,load_model

# sklearn packages
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier   
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,BaggingClassifier,GradientBoostingClassifier
from sklearn import metrics,preprocessing,svm
from sklearn.model_selection import train_test_split

    
print_msg("completed importing modules")

print_msg("loading functions")
          
## Functions to loading data, build models and data preprocessing
## New Log table to capture metrics from different model iterations 
try:                              
    len(log)
except:
    print("create log table")
    log_cols = ["groups","model_name","model_column","data_set", "Accuracy","Precision Score","Recall Score","F1-Score","kappa_score"]
    log = pd.DataFrame(columns=log_cols)

# Function to capture metrics in the log table, save table to log.xlsx file with every run
def metric_update(y_test,y_pred):
    global model_column,model_name,data_set
    global log,itr_cnt
    accuracy = metrics.accuracy_score(y_test,y_pred)
    precision = metrics.precision_score(y_test,y_pred,average='macro',labels=np.unique(y_pred))
    recall = metrics.recall_score(y_test,y_pred,average='macro',labels=np.unique(y_pred))
    f1_score = metrics.f1_score(y_test,y_pred,average='macro',labels=np.unique(y_pred))
    kappa_score=metrics.cohen_kappa_score(y_test,y_pred)
    col_data=[msg_grp,model_name,model_column,data_set,accuracy,precision,recall,f1_score,kappa_score]
    log_entry = pd.DataFrame([col_data], columns=log_cols)
    log = log.append(log_entry)
    itr_cnt=itr_cnt+1
    print_msg("completed iteration ={}".format(itr_cnt))
    print_msg(metrics.classification_report(y_test,y_pred))


# Function to load data and perform preprocessing,add new columns in dataframe    
def load_data(file,prefix):        
    print_msg("loading file=",file)
    df = pd.read_excel(file)
    df= df.drop("Caller" , axis=1)
    df.columns=["short_description","long_description","assigned_group"]
    df["combined_description"]=df["short_description"]+" "+df["long_description"]
    df.dropna(inplace=True) ## Not many null ,so can safely drop the rows
    df,name=preprocess_column(df,"short_description")
    df,name=preprocess_column(df,"combined_description")
    df["assigned_group_org"]=df["assigned_group"]
    df.dropna(subset=['combined_description_text'],inplace=True)
    df.short_description_text[df.short_description_text.isnull()]=df.combined_description_text[df.short_description_text.isnull()]
    df.to_excel(prefix+file)
    return df
    

# Data preprocessing 
# When we set the flag deacc=True , the function removes punctuations also 
def sent_to_words(sentences):
    for sentence in sentences:
        yield(simple_preprocess(str(sentence), deacc=True , min_len=3, max_len=20))  # deacc=True removes punctuations

# covert list to sentences
def words_to_sent(sentences):
    final=[]
    for sentence in sentences:
        local=" ".join(sentence)
        final.append(local)
    return final

## Stopword ,duplicate word removal
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'received'])

def remove_stopwords_duplicate(texts):
    final=[]
    for doc in texts:
        local=[]
        for word in simple_preprocess(str(doc)):
            if word not in local:
                local.append(word)
        final.append(local)
    return final

## find POS for each word for lemmatizer
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN,  "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def preprocess_document(documents):
    ## Convert to lower case
    documents = [sent.lower() for sent in documents]
    # Remove Emails
    documents = [re.sub('\S*@\S*\s?', ' ', sent) for sent in documents]
    # Remove new line characters
    documents = [re.sub('\s+', ' ', sent) for sent in documents]
    # Remove _
    documents = [re.sub('_', ' ', sent) for sent in documents]
    # Remove Numbers
    documents = [re.sub('\d+', ' ', sent) for sent in documents]
    # Remove  distracting single quotes
    documents = [re.sub('\'', ' ', sent) for sent in documents]
    # Remove all non word characters
    documents = [re.sub('\W', ' ', sent) for sent in documents]
    
    document_words = list(sent_to_words(documents))
    document_words = remove_stopwords_duplicate(document_words)
    
    # Init Lemmatizer
    lemmatizer = WordNetLemmatizer()
#     print_msg("lemmatization started")
    hl_lemmatized = []
    for tokens in document_words:
        lemm = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in tokens]
        hl_lemmatized.append(lemm)
    document_words=hl_lemmatized   
#     print_msg("lemmatization ended")
    
    return document_words

def preprocess_column(df,column_name):
    new_column=column_name+"_list"
    new_column1=column_name+"_text"
    try:
        df[new_column].shape
        print_msg("pre-processing was already done")
    except:
        print_msg("process started for "+column_name)
        documents = df[column_name].values.tolist()
        document_words=preprocess_document(documents)
        df[new_column]=document_words
        df[new_column1]=words_to_sent(document_words)
        print_msg("process finished for "+column_name)
    return df,new_column

## sklearn models with default parameters
def sklearn_model(column_name):
    global model_column
    global model_name
    global data_set
    print_msg("fitting all classic models with column_name=",column_name)
    model_column=column_name
    
    X = df[model_column]
    y = df.assigned_group.astype('category')
 
    vectorizer = CountVectorizer()
    X_bow = vectorizer.fit_transform(X)
    
    tfidf_transformer = TfidfTransformer()
    X_tfidf = tfidf_transformer.fit_transform(X_bow)

    model = LogisticRegression()
    model_name = "LogisticRegression"
    sklearn_model_fit(model,X_bow,X_tfidf,y)

    model = DecisionTreeClassifier(criterion = 'entropy' )
    model_name ="DecisionTreeClassifier"
    sklearn_model_fit(model,X_bow,X_tfidf,y)

    model= RandomForestClassifier()
    model_name = "RandomForestClassifier"
    sklearn_model_fit(model,X_bow,X_tfidf,y)

    model = AdaBoostClassifier(n_estimators= 20)
    model_name = "AdaBoostClassifier"
    sklearn_model_fit(model,X_bow,X_tfidf,y)

    model = BaggingClassifier()
    model_name = "BaggingClassifier"
    sklearn_model_fit(model,X_bow,X_tfidf,y)
   
#     model = GradientBoostingClassifier()
#     name = "GradientBoostingClassifier"
#     classic_model_fit(name,model,X_bow,X_tfidf,y)

    model = GaussianNB()
    X_array_bow  = X_bow.toarray()
    X_array_tfidf  = X_tfidf.toarray()
    model_name = "GaussianNB"
    sklearn_model_fit(model,X_array_bow,X_array_tfidf,y)

    model = svm.SVC()
    model_name = "svm.svc"
    sklearn_model_fit(model,X_bow,X_tfidf,y)
    
def sklearn_model_fit(model,X_bow,X_tfidf,y):
    global model_column,model_name,data_set
    data_set="Bow"
    X_train, X_test, y_train, y_test = train_test_split(X_bow,y, test_size=0.10, random_state=42,stratify=y)
    print_msg("working on",msg_grp,model_column,model_name,data_set)
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    metric_update(y_test,y_pred)
    
    data_set="TFIDF"
    print_msg("working on",msg_grp,model_column,model_name,data_set)
    X_train, X_test, y_train, y_test = train_test_split(X_tfidf,y, test_size=0.10, random_state=42,stratify=y)
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    metric_update(y_test,y_pred) 
    
try:
    len(embeddings)
except:
    embeddings = {}
    
def embedding(glove_file):
    global embeddings
    
    if len(embeddings) < 1000:
        print_msg("embedding length",len(embeddings))
        print_msg("Building Embeddings from "+glove_file)

        for o in open(glove_file,encoding="utf8"):
            word = o.split(" ")[0]
            # print_msg(word)
            embd = o.split(" ")[1:]
            embd = np.asarray(embd, dtype='float32')
            # print_msg(embd)
            embeddings[word] = embd
    else:
        print_msg("Embeddings from ",glove_file,"already exists")
    print_msg("No. of embeddings = ", len(embeddings))  

def keras_model(column_name,epoch=5,fit=0):
    global model_column,model_name,data_set
    model_column=column_name
    model_name="Keras LSTM Model"
    data_set="processed"
    
    print_msg("working on",msg_grp,model_column,model_name,data_set)
    
    max_features = 10000
    #epoch = 20
    batch_size = 100
    
    documents = df[model_column].values.tolist()
    max_allowed=max(df["short_description_text"].apply(lambda x: len(x.split(" "))))*2
    max_allowed=40
    maxlen = max(df[model_column].apply(lambda x: len(x)))
    #print_msg("maxlen before=",maxlen) 
    if maxlen>max_allowed:
        maxlen=max_allowed
    print_msg("maxlen after=",maxlen)    

    tokenizer = Tokenizer(num_words=max_features)
    tokenizer.fit_on_texts(list(documents))
    sequence = tokenizer.texts_to_sequences(documents)
    tokenizer_json = tokenizer.to_json()
    with io.open('tokenizer.json', 'w', encoding='utf-8') as f:
        f.write(json.dumps(tokenizer_json, ensure_ascii=False))
        
    word_index = tokenizer.word_index
    vocab_size = len(word_index)+1
    
    print_msg("My vocabulary size = ",vocab_size)
    X = pad_sequences(sequence, maxlen = maxlen, padding='post',truncating='post') 
    
    groups = list(df.assigned_group.unique())
    with open('group.txt', 'w') as filehandle:
        for listitem in groups:
            filehandle.write('%s\n' % listitem)
        
    le = preprocessing.LabelEncoder()
    le.fit(groups)
    y=to_categorical(le.transform(df.assigned_group))   

    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1, random_state = 42,stratify=y)   
    
    glove_file='glove.6B.200d.txt'
    embedding(glove_file)
    embedding_size=embeddings['the'].shape[0]
    
    embedding_matrix = np.zeros((vocab_size, embedding_size))
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, weights = [embedding_matrix],input_length=maxlen))
    model.add(SpatialDropout1D(0.2))
    model.add(BatchNormalization())
    model.add(Bidirectional(LSTM(df.assigned_group.nunique()*2, return_sequences = True,recurrent_dropout=0.1, dropout=0.1)))
    model.add(GlobalMaxPool1D())
    model.add(Dropout(0.4))
    model.add(BatchNormalization())
    model.add(Dense(df.assigned_group.nunique()*2, activation="relu"))
    model.add(Dropout(0.4))
    model.add(BatchNormalization())
    model.add(Dense(df.assigned_group.nunique(), activation="softmax"))

    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['acc'])
    print_msg(model.summary())
    
    if fit>0:
        print_msg('running model.fit')
        history = model.fit(X_train, y_train, epochs = epoch, batch_size=batch_size, validation_split=0.05,shuffle= True,verbose = 1)
        model.save('my_model.h5')
        y_predict=model.predict_classes(X_test)
        y_test1=np.argmax(y_test,axis=1)
        
        metric_update(y_test1, y_predict)
        
def model_load():
    with open('tokenizer.json') as f:
        data = json.load(f)
    tokenizer = tokenizer_from_json(data)
    model = load_model('my_model.h5')
    
    groups = []
    with open('group.txt', 'r') as filehandle:
        for line in filehandle:
            currentPlace = line[:-1]
            groups.append(currentPlace)
        
    le = preprocessing.LabelEncoder()
    le.fit(groups)
    
    return le,tokenizer,model

def model_predict(text):
    document=[text]
    documents=preprocess_document(document)
#     print_msg(documents)
    sequence = tokenizer.texts_to_sequences(documents)
    maxlen=model.input.shape[1]
    X = pad_sequences(sequence, maxlen = maxlen, padding='post',truncating='post')
#     print_msg(X)
    y=model.predict_classes(X)
    print_msg("text=",document,"       predicted_class=",le.inverse_transform(y))

print_msg("Imported modules and new functions completed")

17:33:24 : importing modules
17:33:24 : completed importing modules
17:33:24 : loading functions
17:33:24 : Imported modules and new functions completed


In [ ]:
df.to_excel("1657data.xlsx",index=False)

In [2]:
os.chdir("C:\\work\\capstone")
file="1657data.xlsx"
df=pd.read_excel(file)
df.short_description_list=list(sent_to_words(df.short_description_text.values.tolist()))
df.combined_description_list=list(sent_to_words(df.combined_description_text.values.tolist()))
# df=load_data("data.xlsx","1657")   # load data and backup processed data into a file prefix with 1657

In [12]:
# start model building with all 74 groups , then reduce the number of groups by aggregating low sample groups into one large group
## Perfrom 3 iteration with all models
itr_cnt=0   
for i in [2,30,100]:  
    df["assigned_group"]=df["assigned_group_org"]
    f=( df.assigned_group.value_counts() < i ) 
    j=f[f].index
    df.assigned_group[df.assigned_group.isin(j)]="GRP_74"
    new_groups=df.assigned_group.nunique()
    print_msg("total count for GRP_74={}".format(new_groups))
    msg_grp="grp_74 with <{} rows, total groups={}".format(i,new_groups)
    model_column=" "
    model_name=" "
    data_set=" "
    sklearn_model("short_description_text")
#     sklearn_model("combined_description_text")
    keras_model("short_description_list",20,1)
#     keras_model("combined_description_list",20,1)
    log.to_excel("log.xlsx")

17:33:31 : total count for GRP_74=69
17:33:31 : fitting all classic models with column_name= short_description_text
17:33:31 : working on grp_74 with <2 rows, total groups=69 short_description_text LogisticRegression Bow


C:\Users\A1011153\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\A1011153\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


17:33:32 : completed iteration =1
17:33:32 :               precision    recall  f1-score   support

       GRP_0       0.72      0.96      0.82       397
       GRP_1       0.00      0.00      0.00         3
      GRP_10       0.71      0.36      0.48        14
      GRP_11       0.00      0.00      0.00         3
      GRP_12       0.65      0.65      0.65        26
      GRP_13       0.42      0.33      0.37        15
      GRP_14       0.40      0.33      0.36        12
      GRP_15       1.00      0.25      0.40         4
      GRP_16       0.00      0.00      0.00         9
      GRP_17       0.89      1.00      0.94         8
      GRP_18       0.50      0.33      0.40         9
      GRP_19       0.50      0.14      0.21        22
       GRP_2       0.55      0.25      0.34        24
      GRP_20       0.00      0.00      0.00         4
      GRP_21       0.00      0.00      0.00         3
      GRP_22       0.00      0.00      0.00         3
      GRP_23       0.00      0.00   

C:\Users\A1011153\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\A1011153\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\A1011153\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


17:33:34 : completed iteration =2
17:33:34 :               precision    recall  f1-score   support

       GRP_0       0.62      0.99      0.76       397
       GRP_1       0.00      0.00      0.00         3
      GRP_10       0.80      0.29      0.42        14
      GRP_11       0.00      0.00      0.00         3
      GRP_12       0.69      0.42      0.52        26
      GRP_13       0.44      0.27      0.33        15
      GRP_14       0.80      0.33      0.47        12
      GRP_15       0.00      0.00      0.00         4
      GRP_16       1.00      0.11      0.20         9
      GRP_17       0.89      1.00      0.94         8
      GRP_18       0.00      0.00      0.00         9
      GRP_19       0.00      0.00      0.00        22
       GRP_2       0.33      0.08      0.13        24
      GRP_20       0.00      0.00      0.00         4
      GRP_21       0.00      0.00      0.00         3
      GRP_22       0.00      0.00      0.00         3
      GRP_23       1.00      0.50   

...

C:\Users\A1011153\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\A1011153\AppData\Roaming\Python\Python37\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


17:56:35 : completed iteration =44
17:56:35 :               precision    recall  f1-score   support

       GRP_0       0.47      1.00      0.64       397
      GRP_10       0.00      0.00      0.00        14
      GRP_12       0.00      0.00      0.00        26
      GRP_13       0.00      0.00      0.00        14
      GRP_14       0.00      0.00      0.00        12
      GRP_19       0.00      0.00      0.00        22
       GRP_2       0.00      0.00      0.00        24
      GRP_24       0.00      0.00      0.00        29
      GRP_25       0.00      0.00      0.00        12
       GRP_3       0.00      0.00      0.00        20
      GRP_33       0.00      0.00      0.00        11
       GRP_4       0.00      0.00      0.00        10
       GRP_5       0.00      0.00      0.00        13
       GRP_6       0.00      0.00      0.00        18
      GRP_74       0.00      0.00      0.00       136
       GRP_8       0.00      0.00      0.00        66
       GRP_9       0.00      0.00  

C:\Users\A1011153\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


17:56:35 : maxlen after= 40
17:56:35 : My vocabulary size =  5258
17:56:35 : Embeddings from  glove.6B.200d.txt already exists
17:56:35 : No. of embeddings =  400000
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 40, 200)           1051600   
_________________________________________________________________
spatial_dropout1d_4 (Spatial (None, 40, 200)           0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 40, 200)           800       
_________________________________________________________________
bidirectional_4 (Bidirection (None, 40, 68)            63920     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 68)                0         
_________________________________________________________________
dropout_8 (Dropout) 

C:\Users\A1011153\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [22]:
log.sort_values(by=['kappa_score'],ascending=False).head(10) 

,groups,model_name,model_column,data_set,Accuracy,Precision Score,Recall Score,F1-Score,kappa_score
0,"grp_74 with <2 rows, total groups=69",Keras LSTM Model,short_description_list,processed,0.677267,0.525336,0.395995,0.427204,0.552353
0,"grp_74 with <2 rows, total groups=69",LogisticRegression,short_description_text,Bow,0.676090,0.590135,0.371567,0.426145,0.531342
0,"grp_74 with <2 rows, total groups=69",LogisticRegression,short_description_text,Bow,0.676090,0.590135,0.371567,0.426145,0.531342
0,"grp_74 with <100 rows, total groups=17",BaggingClassifier,short_description_text,TFIDF,0.663133,0.537706,0.393470,0.430700,0.523767
0,"grp_74 with <30 rows, total groups=36",LogisticRegression,short_description_text,Bow,0.669022,0.732875,0.400116,0.479534,0.518314
0,"grp_74 with <30 rows, total groups=36",LogisticRegression,short_description_text,Bow,0.669022,0.732875,0.400116,0.479534,0.518314
0,"grp_74 with <100 rows, total groups=17",LogisticRegression,short_description_text,Bow,0.664311,0.582486,0.355186,0.403217,0.511237
0,"grp_74 with <2 rows, total groups=69",Keras LSTM Model,short_description_list,processed,0.641932,0.489569,0.404149,0.420200,0.508811
0,"grp_74 with <30 rows, total groups=36",Keras LSTM Model,short_description_list,processed,0.656066,0.574363,0.389137,0.422317,0.506353
0,"grp_74 with <100 rows, total groups=17",Keras LSTM Model,short_description_list,processed,0.664311,0.686335,0.380400,0.415979,0.504596


In [23]:
os.chdir("C:\\work\\capstone")
le,tokenizer,model=model_load()

In [24]:
model_predict("crm add-in is getting disabled from outlook ")

19:58:20 : text= ['crm add-in is getting disabled from outlook ']        predicted_class= ['GRP_0']


In [25]:
log.shape

(74, 9)

## Best model out of all iterations coming with LSTM with 69 group, Kappa_score=55%

Summary - 
Not sufficient data to build a model with high accuracy and Kappa score.  
Most of the classes are unbalanced.  
Even after grouping unbalanced classes into large group best accuracy achieved was 69% and kappa_score=55%  
Furhter data cleanup, Removal of duplicate words etc improved the score by 1% level only.  
Keras deep model with regularization and drop layer improved the scope atleast by 2-3%.  
Total 74 iterations are performed with multiple models and tuning parameters.  


# As a next step lets try LDA model to reduce the number of groups so that efficieny can be improved.